In [1]:
import datetime
import numpy as np
import pandas as pd
import random

from scipy.signal import find_peaks

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates
from matplotlib.dates import MONDAY, DateFormatter, DayLocator, WeekdayLocator

%matplotlib inline

import vasu
import vasu_strategy as strategy

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [13]:
def get_trough_indices(dataframe):
    low_price_seq = dataframe['Low Price'].values
    trough_indices = get_troughs(low_price_seq)
    return trough_indices    

def get_peak_indices(dataframe):
    high_price_seq = dataframe['High Price'].values
    peak_indices = get_peaks(high_price_seq)
    return peak_indices    

def get_peaks(price_seq):
    threshold_val = np.mean(price_seq)*0.01
    peaks,peaks_properties = find_peaks(price_seq,distance=4,prominence=5)
    return peaks

def get_troughs(price_seq):
    price_seq_inv = price_seq*-1
    threshold_val = np.mean(price_seq)*0.01
    troughs,troughs_properties = find_peaks(price_seq_inv,distance=4,prominence=5)
    return troughs

def get_momentum_high_df(dataframe):
    peak_indices = get_peak_indices(dataframe)
    high_data = pd.DataFrame(data=[],index = dataframe.index[peak_indices],columns=['high_type'])
    high_data['high_type'] = 'momentum high'
    return high_data

In [3]:
STOCK_NAME = 'ITC'
STOCK_PRICE_FILE = './../database/500875.csv'

In [4]:
price_data = pd.read_csv(STOCK_PRICE_FILE,index_col = 0, parse_dates = True,
                             usecols = [u'Date',u'Open Price', u'High Price', u'Low Price', u'Close Price'],
                             dtype= {u'Open Price':np.float64,u'High Price':np.float64,
                                     u'Low Price':np.float64,u'Close Price':np.float64}).sort_index()

In [5]:
sample_data = price_data.loc[datetime.date(year=2018,month=8,day=1):datetime.date(year=2018,month=12,day=1)]

### High and Lows

In [24]:
momentum_high_data = get_momentum_high_df(sample_data)
#sample_data.join(momentu_high_data)
momentum_high_data

,high_type
Date,
2018-09-03,momentum high
2018-10-01,momentum high
2018-10-22,momentum high
2018-11-05,momentum high


In [87]:
higher_high_index = []
related_high_dates = []

for momentum_high_ts in momentum_high_data.index:
    high_price = sample_data[u'High Price'].loc[momentum_high_ts]
    select_data = sample_data.loc[momentum_high_ts:]
    higher_high_data = select_data[select_data[u'Close Price'] > high_price][0:1]
    if len(higher_high_data) > 0:
        higher_high_index.append(higher_high_data.index[0])
        related_high_dates.append(momentum_high_ts)   

In [88]:
higher_high_index

[Timestamp('2018-11-29 00:00:00')]

In [89]:
higher_high_data = pd.DataFrame(data=related_high_dates, index=higher_high_index,columns=['related high date'])
higher_high_data['high_type'] = 'higher_high'

In [90]:
pd.merge(higher_high_data, momentum_high_data,right_index=True, left_index=True,on=['high_type'],how='outer')

,related high date,high_type
2018-09-03,NaT,momentum high
2018-10-01,NaT,momentum high
2018-10-22,NaT,momentum high
2018-11-05,NaT,momentum high
2018-11-29,2018-11-05,higher_high


In [69]:
momentum_high_data.index

DatetimeIndex(['2018-09-03', '2018-10-01', '2018-10-22', '2018-11-05'], dtype='datetime64[ns]', name='Date', freq=None)

In [84]:
higher_high_index[0][0]

numpy.datetime64('2018-11-29T00:00:00.000000000')

In [22]:
sample_data[momentum_high_data.index[0+1]:]

,Open Price,High Price,Low Price,Close Price
Date,,,,
2018-10-01,300.65,302.95,293.50,296.70
2018-10-03,299.70,299.70,294.00,294.60
2018-10-04,292.50,295.65,285.55,287.25
2018-10-05,287.00,287.55,275.95,277.15
2018-10-08,279.30,279.30,269.55,273.45
2018-10-09,276.00,276.00,267.30,268.50
2018-10-10,270.00,271.30,267.35,270.05
2018-10-11,265.55,269.25,263.60,265.20
2018-10-12,268.90,277.50,267.75,274.70


In [7]:
peak_indices = sample_data[u'High Price'][peak_indices]
momentum_high_df = pd.DataFrame(peak_values.values,index = sample_data.index[peak_indices],columns=['high'])
momentum_high_df

NameError: name 'peak_indices' is not defined

### Detect hammer

In [6]:
def is_hammer(row):
    candle_body = abs(row['Open Price'] - row['Close Price'])
    candle_length = row['High Price'] - row['Low Price']
    if (row['Close Price'] > row['Open Price']):
        candle_upper_shadow = row['High Price'] - row['Close Price']
    else:
        candle_upper_shadow = row['High Price'] - row['Open Price']
        
    if candle_body <= 0.3*candle_length and candle_upper_shadow <= 0.15*candle_length:
        return True
    else:
        return False

In [7]:
sample_data[sample_data.apply(is_hammer,axis=1)]

,Open Price,High Price,Low Price,Close Price
Date,,,,
2018-09-06,311.0,311.5,308.10,310.70
2018-10-16,283.6,283.8,276.60,282.55
2018-10-31,280.9,280.9,273.15,279.80


In [9]:
vasu.get_trend_data(sample_data)

,Open Price,High Price,Low Price,Close Price,high_slope,low_slope,trend
Date,,,,,,,
2018-05-02,281.50,288.40,280.50,287.15,0.000000,0.000000,indeterminate
2018-05-03,289.00,289.55,282.35,284.25,0.000000,0.000000,indeterminate
2018-05-04,286.50,286.50,276.00,277.50,0.000000,0.000000,indeterminate
2018-05-07,278.00,282.85,276.10,281.90,0.000000,-0.413217,sideways
2018-05-08,283.00,285.85,281.60,282.35,0.000000,-0.413217,sideways
2018-05-09,282.00,284.00,281.40,282.25,0.000000,-0.413217,sideways
2018-05-10,281.60,283.85,277.50,279.85,0.000000,-0.413217,sideways
2018-05-11,281.00,284.65,279.85,283.90,0.000000,-0.413217,sideways
2018-05-14,282.05,287.20,282.05,284.45,0.000000,-0.413217,sideways
